# 1. STFT变换

## 1.1 振动（原始）信号的波形及功率谱函数

In [ ]:
import librosa
import numpy as np
import matplotlib.pyplot as plt
import librosa.display

#加载信号
signal,sample_rate = librosa.load()

#信号补零 
frame_size,hop_size=256,64
if len(signal) % hop_size != 0:
    frame_num = int((len(signal)-frame_size)/hop_size)+1
    pad_num = frame_num * hop_size +frame_size - len(signal)
    signal = np.pad(signal,pad_width=(0,pad_num),mode='wrap')
frame_num = int((len(signal)-frame_size)/hop_size)+1

#信号分帧
row = np.tile(np.arange(0,frame_size),(frame_num,1))
column = np.tile(np.arange(0,frame_num * hop_size,hop_size),(frame_size,1)).T
index = row + column
signal_frame = signal[index]

#加汉明窗
signal_frame = signal_frame * np.hanming(frame_size)

#对信号做傅里叶变换
n_fft = 256  #输出频率点为 (n_fft/2)+1
signal_stft = np.fft.rfft(signal_frame,n_fft)

#计算功率谱函数和分贝值
signal_pow = np.abs(inform_stft)**2/n_fft
signal_db = 20*np.log10(signal_pow)

#绘制波形
plt.figure(figsize = (20,10))
librosa.display.waveshow(signal,color='blue')
plt.imshow(signal_db)
y_ticks = np.arange(0,int(n_fft/2),100)
plt.yticks(y_ticks,labels = y_ticks*sample_rate/n_fft)
plt.show()

## 1.2 添加不同的噪声水平

In [ ]:
#为信号添加噪声，根据给定的信噪比（SNR）生成噪声
import scipy.signal as signal

#生成粉红噪声  粉红噪声和频率成反比
def pink_noise(size):
    #使用Scipy的butter滤波器生成粉红噪声
    white_noise = np.random.normal(0,1,size)
    #采用高频滤波器
    b,a = signal.butter(1,0.05,'high')
    pink = signal.filtfilt(b,a,white_noise)
    #归一化
    return pink / np.max(np.abs(pink))
    

def add_noise_to_signal(signal,noise_level_percentage,noise_type='white ',size=None):
    #如果没有指定size 则用信号的长度
    if size is None:
        size = len(signal)
        
    #计算信号的最大幅度
    max_signal_amplitude = np.max(np.abs(signal))
    
    #根据噪声水平百分比计算噪声标准差
    noise_std = max_signal_amplitude * (noise_level_percentage / 100)
    
    #生成噪声 根据噪声类型选择
    if noise_type == 'white':
        noise = np.random.normal(0,noise_std,size)
    elif noise_type == 'pink':
        pink = pink_noise(size)
        noise = pink * noise_std #以噪声标准差来调整幅度
    else:
        raise ValueError("Unsupported noise type. Choose 'white' or 'pink'.")
        
   
    return noise

#输入白噪声和粉红噪声的数据集大小
white_noise_size = 190218
pink_noise_size = 190225

#噪声水平的百分比
noise_level_percentage = [20,40,60,80]

#生成白噪声和粉红噪声
for noise_level in noise_level_percentage:
    white_noise = add_noise_to_signal(signal,noise_level_percentage,noise_type='white',size=white_noise_size)
    pink_noise = add_noise_to_signal(signal,noise_level_percentage,noise_type='pink',size=pink_noise_size)
    
    
#将噪声添加到原始信号中
noisy_signal_white = signal[:white_noise_size] + white_noise
noisy_signal_pink = signal[:pink_noise_size] + pink_noise

#输出噪声信号
print("White Noise Signal Length:", len(noisy_signal_white))
print("Pink Noise Signal Length:", len(noisy_signal_pink))

#绘制噪声波形
plt.figure(figsize = (20,10))
librosa.display.waveshow(noisy_signal_white,color='Accent')
librosa.display.waveshow(noisy_signal_pink,color='gist_ncar')

 

# 2.卷积神经网络（CNN）

## 2.1构建神经网络模型

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim

#定义CNN模型（因为没有看到数据的变化  不知道在哪一步进行了最大池化）
class DenoiserCNN(nn.Module):
    def _init_(self,input_channels=1,output_channels=1):
        super(DenoiserCNN,self)._init_()
        
        #第1层 zeropadding  conv2D  Relu batchnorm  maxpooling
        self.layer1 = nn.Sequential(
        #填充0，使得卷积不改变宽度
        nn.ZeroPad2d((4,4,0,0)),
        nn.Conv2d(input_channels,18,kernel_size=(3,3),stride=3),
        #激活函数
        nn.ReLU(),
        #归一化
        nn.BatchNorm2d(18)
        )
        
        #第2层  conv2D  Relu batchnorm 
        self.layer2 = nn.Sequential(
        nn.Conv2d(18,36,kernel_size=(3,3),stride=1,padding=1),
        nn.ReLU(),
        nn.BatchNorm2d(36)
        )
        
        #第3层  conv2D  Relu batchnorm
        self.layer3 = nn.Sequential(
        nn.Conv2d(36,64,kernel_size=(3,3),stride=1,padding=1),
        nn.ReLU(),
        nn.BatchNorm2d(64)
        )
        
        #................................
        
        #第14层  conv2D Relu batchnorm  
        self.layer14 = nn.Sequential(
        nn.Conv2d(64,8,kernel_size=(3,3),stride=1,padding=1),
        nn.ReLU()
        nn.BatchNorm2d(8)
        )
        
        #第15层卷积  conv2D Relu batchnorm
        self.layer15 = nn.Sequential(
        nn.Conv2d(8,1,kernel_size=(3,3),stride=1,padding=1),
        nn.ReLU()
        nn.BatchNorm2d(1)
        )
        
        #输出去噪信号
        self.output_layer = nn.Conv2d(1,output_channels,kernel_size=(1,1))
        
    def forward(self,x):
        x = self.layer1(x)
        x = self.layer2(x)
        x = self.layer3(x)
        #.........................
        x = self.layer14(x)
        x = self.layer15(x)
        x = self.output_layer(x)
        return x 
    
#初始化模型
model = DenoiserCNN()

#定义损失函数和优化器
#使用MSE和RMSE作为损失函数  采用Adam优化器 学习率为0.0003
criterion_RMSE = nn.RMSELoss()
criterion_MSE = nn.MSELoss()
optimizer = optim.Adam(model.parameters(),lr=0.0003)

## 2.2 训练模型

In [ ]:
#输入噪声信号和目标信号（即振动信号） 假设迭代次数为100
for epoch in range(100):
    for noisy_signal_white, noisy_signal_pink, signal in zip(noisy_signal_white, noisy_signal_pink, signal):
        #添加批次和通道维度  将numpy数组或python列表转换为pytorch浮点数张量  .unsqueeze(0)：向张量添加新的维度，调用两次最终将数据的形状调整为 (batch_size, channels, depth, height, width)。
        noisy_signal_white_tensor = torch.tensor(noisy_signal_white).float().unsqueeze(0).unsqueeze(0)
        noisy_signal_pink_tensor = torch.tensor(noisy_signal_pink).float().unsqueeze(0).unsqueeze(0)
        target_signal_tensor = torch.tensor(signal).float().unsqueeze(0).unsqueeze(0)
        
        #前向传播
        output_signal_white = model(noisy_signal_white_tensor) 
        output_signal_pink = model(noisy_signal_pink_tensor) 
        
        #计算损失
        loss_white_RMSE = criterion_RMSE(output_signal_white,target_signal_tensor)
        loss_white_MSE = criterion_MSE(output_signal_white,target_signal_tensor)
        loss_pink_RMSE = criterion_RMSE(output_signal_pink,target_signal_tensor)
        loss_pink_MSE = criterion_MSE(output_signal_pink,target_signal_tensor)
        
        #反向传播和优化
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
    print(f'Epoch{epoch+1},Loss:{loss.item()}')
        

# 3.通过线性参考指标和非线性系统指标评估

## 3.1 线性评估指标计算

In [ ]:
#计算SNR
def compute_snr(target_signal,output_signal_white,output_signal_pink):
    noise_white = target_signal - output_signal_white
    noise_pink = target_signal - output_signal_pink
    snr_white = 10 * torch.log10(torch.sum(target_signal**2)/torch.sum(noise_white**2))
    snr_pink = 10 * torch.log10(torch.sum(target_signal**2)/torch.sum(noise_pink**2))
    return snr_white,snr_pink

#计算RMSE
def compute_rmse(target_signal,output_signal_white,output_signal_pink):
    rmse_white = torch.sqrt(torch.mean((target_signal - output_signal_white)**2))
    rmse_pink = torch.sqrt(torch.mean((target_signal - output_signal_pink)**2))
    return rmse_white,rmse_pink


## 3.2 非线性系统指标

### 3.2.1 小波变换

In [ ]:
import pywt

#使用db8小波函数分解5层
def wavelet_denoising(noisy_signal,wavelet='db8',level=5):
    #进行小波分解
    coeffs = pywt.wavedec(noisy_signa,wavelet,level=level)
    #选择阈值
    threshold = np.sqrt(2 * np.log(len(noisy_signal)))*(1 / np.sqrt(2))
    #对小波系数进行阈值处理
    coeffs_filtered = [pywt.threshold(c,thredhold,mode='soft') for c in coeffs]
    #进行逆小波变换，重建去噪信号
    denoised_signal = pywt.waverec(coeffs_filtered,wavelet)
    
    return denoised_signal

#对白噪声和粉红噪声进行小波去噪
denoise_signal_white = wavelet_denoising(noisy_signal_white)
denoise_signal_pink = wavelet_denoising(noisy_signal_pink)

#绘制原始信号和去噪后的信号（以白噪声为例）
plt.figure(figsize=(10,6))
plt.plot(noisy_signal_white,label='noisy_signal_White',color='blue')
plt.plot(denoise_signal_white,label='denoised_signal_White',color='red')
plt.legend()
plt.xlabel('Sampling')
plt.ylabel('Amplitude')
plt.show()

### 3.2.2 振动信号的维度曲线、分形吸引子和Poincare曲线(这里我只画了一个）

In [ ]:
from scipy.spatial.distance import pdist,squareform
from scipy.integrate import odeint

#使用延迟嵌入方法重构信号的相空间   embedding_dim: 嵌入维度  tau: 延迟时间
def reconstruct_phase_space(signal, embedding_dim, tau):
    N = len(signal)
    #切片并遍历所有的值 间隔延迟时间取嵌入维度的值组成一个数组
    reconstructed_space = np.array([signal[i:i + embedding_dim * tau:tau] for i in range(N - embedding_dim * tau)])
    return reconstructed_space

#嵌入维度有三种方法  文章中采取的是经验选择 选择的是5 10 15 20
embedding_dim = [5,10,15,20]

#计算相关维度 阈值epsilon设置为0.1
def correlation_dimension(signal, embedding_dim, tau, epsilon=0.1):
    reconstructed_space = reconstruct_phase_space(signal, embedding_dim, tau)
    #计算相空间点的欧氏距离并转换为二维方阵
    dist_matrix = squareform(pdist(reconstructed_space,'euclidean'))
    count = np.sum(dist_matrix < epsilon)
    correlation = count / (len(signal) ** 2)
    return correlation

#定义Rossler系统的方程
def rossler_system(X, t, a, b, c):
    x, y, z = X
    dxdt = -y -z
    dydt = x + a * y
    dzdt = b + z * (x-c)
    return [dxdt, dydt, dzdt]

#Poincare映射 period为周期 设置截面位置的阈值为0.1
def poincare_mapping(signal, period, threshold = 0.1):
    mapping_points = []
    for i in range(1, len(signal) - period):
        if signal[i] < threshold and signal[i+1] >= threshold:
            mapping_points.append(signal[i + period])
    return np.array(mapping_points)

#假设延迟时间为1 计算相关维度
tau = 1
correlation = correlation_dimension(signal,embedding_dim,tau,epsilon)

#绘制相关维度
plt.figure(figsize=(10,6))
plt.plot(correlation)
plt.xlabel('Time Step')
plt.ylabel('Correlation')
plt.show()

#设置Rossler系统的参数
a, b, c = 0.2, 0.2, 5.7
X0 = [1,0,0] #初始条件
t = np.linspace(0, 100, 10000)#时间范围

#求解Rossler系统
solution = odeint(rossler_system,X0,t,args = (a,b,c))

#提取x, y, z
x, y, z = solution[:,0], solution[:,1], solution[:,2]

#绘制Rossler系统的分形吸引子
fig = plt.figure(figsize=(10,6))
ax = fig.add_subplot(111,projection='3d')#1行1列1个子图采用3D绘图
ax.plot(x,y,z,lw=0.5)
ax.set_tittle("Fractal Attractor of Rossler")
ax.set_xlabel('X')
ax.set_ylabel('Y')
ax.set_zlabel('Z')
plt.show()

#提取poincare映射（因为频率为100  故设置每个周期有100个采样点）
period = 100
poincare_points = poincare_mapping(signal,period)

#绘制Poincare映射   #不过不知道提取的是哪个面上的映射
plt.figure(figsize=(10,6))
plt.scatter(poincare_points[:, 0], poincare_points[:, 1], s=1, color='black')  # 使用scatter绘制Poincaré映射 提取x坐标和y坐标 每个点的大小为1像素
plt.xlabel('X')
plt.ylabel('Y')
plt.title('Poincare Map of Rossler')
plt.show()


# 4.协方差驱动的随机子空间法进行模态识别

In [ ]:
#假设signal_1和signal_2是传感器上两个模态的振动信号
#生成包含两个传感器的信号数据
signal_m =np.vstack([signal_1,signal_2])

#计算协方差矩阵
cov_matrix = np.cov(signal_m)

#随机子空间法和特征值分解
#求解协方差矩阵的特征值和特征向量
eigvals, eigvecs = np.linalg.eig(cov_matrix)

# 输出特征值（表示不同模态的频率）
print("Eigenvalues:", eigvals)
print("Eigenvectors:", eigvecs)

# 选择特征值中的最大值作为固有频率
modal_frequencies = np.sqrt(eigvals)  # 将特征值取平方根得到频率
print("Modal Frequencies (Hz):", modal_frequencies)

# 计算阻尼比
# 这里用特征向量的虚部/实部作为阻尼比的一个简单近似
damping_ratios = np.abs(np.imag(eigvecs)) / np.abs(np.real(eigvecs))

#绘制稳定图  
plt.figure(figsize=(8, 6))
plt.scatter(modal_frequencies, damping_ratios, color='red', label='Modal Frequencies')
plt.xlabel('Frequency (Hz)')
plt.ylabel('Stability (arbitrary units)')
plt.title('Stability Plot')
plt.legend()
plt.grid(True)
plt.show()